# Bangalore Traffic Density Analysis

This notebook focuses on analyzing traffic density specifically within Bangalore city limits. This targeted approach helps reduce storage and processing requirements while providing more accurate predictions for the local traffic patterns.

In [ ]:
# Define Bangalore city boundaries
BANGALORE_BOUNDS = {
    'north': 13.023577,
    'south': 12.923210,
    'east': 77.642256,
    'west': 77.538899
}

def is_within_bangalore(lat, lon):
    """Check if coordinates are within Bangalore city limits"""
    return (BANGALORE_BOUNDS['south'] <= lat <= BANGALORE_BOUNDS['north'] and
            BANGALORE_BOUNDS['west'] <= lon <= BANGALORE_BOUNDS['east'])

# Filter function for traffic data
def filter_bangalore_data(df):
    """Filter traffic data to only include points within Bangalore"""
    return df[df.apply(lambda row: is_within_bangalore(row['latitude'], row['longitude']), axis=1)]

## Bangalore-Specific Traffic Patterns

Bangalore has unique traffic patterns due to:
1. High concentration of IT corridors
2. Metro construction zones
3. Peak hours different from other cities (due to IT industry)
4. Specific congestion hotspots

We'll adjust our analysis to account for these factors.

# Traffic Density Analysis for Vehicle Safety

This notebook implements traffic density estimation using YOLOv8 for improving vehicle safety predictions in the ORBITRIX app. We'll use data from Kaggle to train and evaluate our model.

## 1. Kaggle API Setup

First, we'll set up the Kaggle API to access the dataset. You'll need to:
1. Install the Kaggle package
2. Set up your Kaggle API credentials

In [ ]:
# Install required packages
!pip install kaggle
!pip install ultralytics
!pip install opencv-python-headless
!pip install pandas numpy matplotlib seaborn

In [ ]:
import os
from pathlib import Path

# Set up Kaggle credentials
# You need to place your kaggle.json file in ~/.kaggle/
kaggle_path = Path.home() / '.kaggle'
kaggle_path.mkdir(exist_ok=True)

# Note: You need to download kaggle.json from your Kaggle account settings
# and place it in the ~/.kaggle directory
print("Please ensure you have placed your kaggle.json file in:", kaggle_path)

## 2. Download Traffic Dataset

Now we'll download the traffic dataset from Kaggle. We'll use a dataset that contains traffic video footage for density estimation.

In [ ]:
# Download dataset
!kaggle datasets download -d farzadnekouei/real-time-traffic-density-estimation-with-yolov8

# Create data directory and extract dataset
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)
!unzip real-time-traffic-density-estimation-with-yolov8.zip -d data/

## 3. Data Preprocessing

Let's prepare our data for the YOLOv8 model by implementing preprocessing functions for video frames.

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

def preprocess_frame(frame, target_size=(640, 640)):
    """Preprocess a single frame for YOLOv8"""
    # Resize frame
    frame = cv2.resize(frame, target_size)
    
    # Convert BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Normalize pixel values
    frame = frame / 255.0
    
    return frame.astype(np.float32)

## 4. YOLOv8 Model Integration

Now we'll set up the YOLOv8 model for traffic detection. We'll use a pre-trained model and fine-tune it for our specific use case.

In [ ]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Load the smallest YOLOv8 model

# Define vehicle classes of interest (COCO dataset indices)
VEHICLE_CLASSES = [2, 3, 5, 7]  # car, motorcycle, bus, truck

def process_detections(results, conf_threshold=0.25):
    """Process YOLOv8 detections and filter vehicle classes"""
    vehicles = []
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            
            if cls in VEHICLE_CLASSES and conf > conf_threshold:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                vehicles.append({
                    'class': cls,
                    'confidence': conf,
                    'bbox': [x1, y1, x2, y2]
                })
    
    return vehicles

## 5. Traffic Density Analysis

Let's implement functions to analyze traffic density based on the detected vehicles.

In [ ]:
def calculate_traffic_density(vehicles, frame_area):
    """Calculate traffic density based on detected vehicles"""
    # Calculate total area occupied by vehicles
    vehicle_area = 0
    for vehicle in vehicles:
        bbox = vehicle['bbox']
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        vehicle_area += width * height
    
    # Calculate density as percentage of frame covered by vehicles
    density = (vehicle_area / frame_area) * 100
    
    return {
        'vehicle_count': len(vehicles),
        'density_percentage': density,
        'congestion_level': get_congestion_level(density)
    }

def get_congestion_level(density):
    """Determine traffic congestion level based on density"""
    if density < 10:
        return "Low"
    elif density < 25:
        return "Moderate"
    else:
        return "High"

## 6. Model Evaluation

Let's evaluate our traffic density estimation model using some test videos.

In [ ]:
def evaluate_model(video_path):
    """Evaluate model performance on a test video"""
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    total_density = 0
    results = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Preprocess frame
        processed_frame = preprocess_frame(frame)
        
        # Run detection
        detections = model(processed_frame)
        vehicles = process_detections(detections)
        
        # Calculate density
        frame_area = frame.shape[0] * frame.shape[1]
        density_info = calculate_traffic_density(vehicles, frame_area)
        
        results.append(density_info)
        total_density += density_info['density_percentage']
        frame_count += 1
    
    cap.release()
    
    # Calculate average metrics
    avg_density = total_density / frame_count if frame_count > 0 else 0
    
    return {
        'average_density': avg_density,
        'frame_count': frame_count,
        'detailed_results': results
    }

# Example usage
test_video = 'data/test_video.mp4'  # Update with your test video path
if os.path.exists(test_video):
    results = evaluate_model(test_video)
    print(f"Average Traffic Density: {results['average_density']:.2f}%")
    print(f"Frames Processed: {results['frame_count']}")